<a href="https://www.kaggle.com/code/aisuko/automatic-speech-recognition-asr?scriptVersionId=164810189" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Automatic speech recognition (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Many of virtual assistants like Siri and Alexa use ASR model to help users everyday, ans there are many other applications like live captioning and note-taking during meetings. Let's finetune Wav2Vec2-base model which pretrained on 16kHz sampled speech audio with `Automatic Speech Recognition` label dataset. We need to make sure the speech input is also sampled at 16kHz.

In [1]:
%%capture
!pip install transformers==4.35.2
!pip install datasets==2.15.0
!pip install evaluate==0.4.1
!pip install jiwer==3.0.3

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# checking Huggingface services status if the login was failed
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning Wav2Vec2-base"
os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
os.environ["WANDB_NAME"] = "ft-wav2vec2-with-minds-asr"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading MinDS-14


Let's pick up a smaller subset of the MInDS-14.

In [3]:
from datasets import load_dataset, Audio

minds=load_dataset("PolyAI/minds14", name="en-US", split="train[:500]")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split: 0 examples [00:00, ? examples/s]

Split the dataset's train split into a train and test set with the `train_test_split` method

In [4]:
minds=minds.train_test_split(test_size=0.2)
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 400
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 100
    })
})

Let's focus on the audio and transcription, and remove the other columns with the `remove_columns` method.

In [5]:
minds=minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [6]:
minds["train"][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~CASH_DEPOSIT/602bacea963e11ccd901cdc3.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~CASH_DEPOSIT/602bacea963e11ccd901cdc3.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00024414,
         -0.00048828, -0.00024414]),
  'sampling_rate': 8000},
 'transcription': 'where can I want to where can I go if I want to deposit cash into my account'}

There are two fileds:

* `audio`: a l-dimensional `array` of the speech signal that must be called to load and resample the audio file.
* `transcription`: the target text

# Preprocess

Let's load a Wav2Vec2 process to process the audio signal with `AutoProcessor` It is multimodel tasks require a processor that combines two types of preprocessing tools.

In [7]:
from transformers import AutoProcessor

processor=AutoProcessor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Let's resample the dataset to 16000kHz to use the pretrained Wav2Vec2 model:

In [8]:
minds=minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~CASH_DEPOSIT/602bacea963e11ccd901cdc3.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~CASH_DEPOSIT/602bacea963e11ccd901cdc3.wav',
  'array': array([-1.49744446e-05, -7.16391878e-06,  1.66330792e-05, ...,
         -4.62295808e-04, -2.78127758e-04, -5.03417286e-05]),
  'sampling_rate': 16000},
 'transcription': 'where can I want to where can I go if I want to deposit cash into my account'}

As we can see in the `transcription` above, the text contains a mix of upper and lowercase characters. The Wav2Vec2 **tokenizer** is only trained on uppercase characters. So we will need to make sure the text maches the tokenizer's vocabulary:

In [9]:
def uppercase(example):
    return {"transcription": example["transcription"].upper()}

minds=minds.map(uppercase)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Let's create a preprocessing function that:
1. Calls the `audio` column to load and resample the audio file
2. Extracts the `input_values` from the audio file and tokenize the transcription column with the processor.

In [10]:
def prepare_dataset(batch):
    audio=batch["audio"]
    batch=processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"]=len(batch["input_values"][0])
    return batch

encoded_minds=minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

We need to adapt the DataCollatorWithPadding to create a batch of examples. It will also dynamically pad our text and lables to the length of the longest element in its batch(instead of the entire dataset) so they are a uniform length. While it is possible to pad our text in the tokenizer function by setting padding=True, dynamic padding is more efficient.

In [11]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str]="longest"
        
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        input_features=[{"input_values": feature["input_values"][0]} for feature in features]
        label_features=[{"input_ids": feature["labels"]} for feature in features]
        
        batch=self.processor.pad(input_features, padding=self.padding, return_tensors="pt")
        
        labels_batch=self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")
        
        # replace padding with -100 to ignore loss correctly
        labels=labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        batch["labels"]=labels
        
        return batch

In [12]:
data_collator=DataCollatorCTCWithPadding(processor=processor, padding="longest")
print(data_collator)

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	2: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	3: AddedToken

# Evaluate

Here we load a evaluation method with Evaluate library. In this notebook, load the word error rate metric:

In [13]:
import evaluate
import numpy as np

wer_eva=evaluate.load("wer")


def compute_metrics(pred):
    pred_logits=pred.predictions
    pred_ids=np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids==-100]=processor.tokenizer.pad_token_id
    
    pred_str=processor.batch_decode(pred_ids)
    label_str=processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer=wer_eva.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

Then create a function that passes our predictions and labels to compute to calcualte the WER:

# Training

In [14]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model=AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

print(model.config)

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "freeze_feat_extract_

In [15]:
training_args=TrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=40,
    max_steps=80,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    save_steps=40,
    eval_steps=20,
    logging_steps=30,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
    push_to_hub=False,   
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: urakiny (causal_language_trainer). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240229_071003-h2osw3p6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-wav2vec2-with-minds-asr
wandb: ⭐️ View project at https://wandb.ai/causal_language_trainer/Fine-tuning%20Wav2Vec2-base
wandb: 🚀 View run at https://wandb.ai/causal_language_trainer/Fine-tuning%20Wav2Vec2-base/runs/h2osw3p6
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Wer
20,No log,48.786919,2.624908
40,56.138600,31.624374,1.022108
60,32.549600,23.657028,1.000000
80,32.549600,22.642813,1.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=80, training_loss=39.02910041809082, metrics={'train_runtime': 1603.8767, 'train_samples_per_second': 1.596, 'train_steps_per_second': 0.05, 'total_flos': 4.478191990104269e+17, 'train_loss': 39.02910041809082, 'epoch': 6.4})

In [16]:
import math

eval_results=trainer.evaluate()
print(f"Perplexity:{math.exp(eval_results['eval_loss']):2f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Perplexity:6817868538.089420


In [17]:
kwargs={
    'model_name': f'{os.getenv("WANDB_NAME")}',
    'finetuned_from': "facebook/wav2vec2-base",
    'tasks': 'automatic-speech-recognition',
    'dataset':'PolyAI/minds14'
}

processor.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

'https://huggingface.co/aisuko/ft-wav2vec2-with-minds-asr/tree/main/'

# Inference

Load an audio file from dataset, and remember to resample the sampling rate of the audio file to match the sampling rate of the model.

In [18]:
from datasets import load_dataset, Audio

dataset=load_dataset("PolyAI/minds14", "en-US", split="train")
dataset=dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate=dataset.features["audio"].sampling_rate
audio_file=dataset[0]["audio"]["path"]

In [19]:
from transformers import pipeline

transcriber=pipeline("automatic-speech-recognition", model=os.getenv("WANDB_NAME"))
transcriber(audio_file)

{'text': 'H'}

## With PyTorch

In [20]:
from transformers import AutoProcessor

processor=AutoProcessor.from_pretrained(os.getenv("WANDB_NAME"))
inputs=processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

In [21]:
from transformers import AutoModelForCTC

model=AutoModelForCTC.from_pretrained(os.getenv("WANDB_NAME"))
with torch.no_grad():
    logits=model(**inputs).logits

In [22]:
import torch

predicted_ids=torch.argmax(logits, dim=-1)
transcription=processor.batch_decode(predicted_ids)
transcription

['H']